# Load in necessary libraries

In [1]:
import pandas as pd
import numpy as np
import time

from scipy.stats import norm, gaussian_kde, percentileofscore

pd.options.display.max_columns = None
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats

# Define function to pull shot chart data from NBA API, specifying player name and season as parameters

In [2]:
def get_player_shotchartdetail(player_name, season_id):
    """
    Parameters
    ----------
    player_name: name of the player with Capital
    season_id: ex. 2012-13
    """
    
    # player dictionary
    nba_players = players.get_players()
    player_dict = [player for player in nba_players if player['full_name'] == player_name][0]
    
    # career df
    career = playercareerstats.PlayerCareerStats(player_id=player_dict['id'])
    career_df = career.get_data_frames()[0]
    
    # team id during the season
    team_id = career_df[career_df['SEASON_ID'] == season_id]['TEAM_ID']
    
    # shotchardtdetail endpoint
    shotchartlist = shotchartdetail.ShotChartDetail(team_id=int(team_id), 
                                                   player_id=int(player_dict['id']), 
                                                   season_type_all_star='Regular Season', 
                                                   season_nullable=season_id,
                                                   context_measure_simple="FGA").get_data_frames()
    
    return shotchartlist[0], shotchartlist[1]

# Example run using Giannis Antetokounmpo during the 2022-23 NBA regular season

In [3]:
player_shotchart_df, league_avg = get_player_shotchartdetail('Giannis Antetokounmpo', '2022-23')
player_shotchart_df

# Create list of all active NBA players for 2022-23 regular season

In [6]:
from nba_api.stats.static  import players
player_dict = players.get_active_players()

In [7]:
player_name = [d['full_name'] for d in player_dict]

player_name

['Precious Achiuwa',
 'Steven Adams',
 'Bam Adebayo',
 'Ochai Agbaji',
 'Santi Aldama',
 'Nickeil Alexander-Walker',
 'Grayson Allen',
 'Jarrett Allen',
 'Jose Alvarado',
 'Kyle Anderson',
 'Giannis Antetokounmpo',
 'Kostas Antetokounmpo',
 'Thanasis Antetokounmpo',
 'Cole Anthony',
 'O.G. Anunoby',
 'Ryan Arcidiacono',
 'Deni Avdija',
 'Deandre Ayton',
 'Udoka Azubuike',
 'Marvin Bagley III',
 'Patrick Baldwin Jr.',
 'LaMelo Ball',
 'LiAngelo Ball',
 'Lonzo Ball',
 'Mo Bamba',
 'Paolo Banchero',
 'Desmond Bane',
 'Dalano Banton',
 'Dominick Barlow',
 'Harrison Barnes',
 'Scottie Barnes',
 'RJ Barrett',
 'Will Barton',
 'Charles Bassey',
 'Keita Bates-Diop',
 'Nicolas Batum',
 'Kent Bazemore',
 'Darius Bazley',
 'Bradley Beal',
 'Malik Beasley',
 'MarJon Beauchamp',
 'Jules Bernard',
 'Davis Bertans',
 'Patrick Beverley',
 'Saddiq Bey',
 'Marcus Bingham Jr.',
 'Khem Birch',
 'Goga Bitadze',
 'Bismack Biyombo',
 'Buddy Boeheim',
 'Bogdan Bogdanovic',
 'Bojan Bogdanovic',
 'Bol Bol',
 'M

# Pull shot chart data for all active NBA players for the 2022-23 NBA regular season from NBA API. Delay was implemented to avoid time out

In [9]:
shotchart = pd.DataFrame()
for i in range(582):
    try:
        player_shotchart_df, league_avg = get_player_shotchartdetail(player_name[i], '2022-23')
        shotchart = pd.concat([shotchart, player_shotchart_df])
        time.sleep(3)
    except (TypeError, IndexError):
        pass
    
shotchart

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022200008,89,1630173,Precious Achiuwa,1610612761,Toronto Raptors,1,4,50,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,27,-159,221,1,0,20221019,TOR,CLE
1,Shot Chart Detail,0022200008,113,1630173,Precious Achiuwa,1610612761,Toronto Raptors,1,3,59,Missed Shot,Running Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,6,22,1,0,20221019,TOR,CLE
2,Shot Chart Detail,0022200008,474,1630173,Precious Achiuwa,1610612761,Toronto Raptors,3,3,8,Missed Shot,Floating Jump shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,8,22,79,1,0,20221019,TOR,CLE
3,Shot Chart Detail,0022200008,511,1630173,Precious Achiuwa,1610612761,Toronto Raptors,3,0,41,Made Shot,Cutting Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-2,21,1,1,20221019,TOR,CLE
4,Shot Chart Detail,0022200008,542,1630173,Precious Achiuwa,1610612761,Toronto Raptors,4,10,18,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,25,-200,153,1,1,20221019,TOR,CLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,Shot Chart Detail,0022201112,36,1627826,Ivica Zubac,1610612746,LA Clippers,1,9,11,Made Shot,Turnaround Hook Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,3,-29,27,1,1,20230325,LAC,NOP
456,Shot Chart Detail,0022201112,56,1627826,Ivica Zubac,1610612746,LA Clippers,1,6,54,Made Shot,Floating Jump shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,4,65,1,1,20230325,LAC,NOP
457,Shot Chart Detail,0022201112,60,1627826,Ivica Zubac,1610612746,LA Clippers,1,6,22,Made Shot,Floating Jump shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,9,-1,90,1,1,20230325,LAC,NOP
458,Shot Chart Detail,0022201112,345,1627826,Ivica Zubac,1610612746,LA Clippers,3,9,31,Made Shot,Cutting Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-9,6,1,1,20230325,LAC,NOP


# Read dataframe of NBA shot chart data into CSV file

In [10]:
shotchart.to_csv("nba_shot_data.csv", index = False)